In [9]:
import pandas as pd
import numpy as np
import random 
from itertools import product
import copy
from tqdm import tqdm
def random_data(): 
    return None
def get_domains(data):
    return None

In [6]:
class Data:

	def generate_random_data(self, variable_data, N=10):

		data = {}
		data['id'] = range(N)
		
		for i, variable in enumerate(variable_data):
			name = variable['name']
			var_type = variable['type']
			step = int(variable['step']) if variable['step'] else 0

			if var_type == 'boolean':
				data[name] = [random.choice([False, True]) for _ in range(N)]
			else:
				min_value = int(variable['min'])
				max_value = int(variable['max'])
				mean = np.random.uniform(min_value, max_value)
				std = np.random.uniform((max_value-min_value)/2000, (max_value-min_value)/100)
				if var_type=='int':
					data[name]  = np.random.normal(mean, std, N).astype(int)
				elif var_type == 'float':
					data[name] = np.random.normal(mean, std, N).round(step)
     
		data['y'] = ['' for _ in range(N)]
		df = pd.DataFrame(data)
		display(df)
		return df

	def __init__(self, variable_data, N):
		self.data = pd.DataFrame(self.generate_random_data(variable_data, N))
		self.__domains = get_domains(self.data)

	def get_values(self, id1, id2=None, X=[]):
		if id2 is None:
			if X == [] or X == self.data.keys():
				return self.data[self.data.id == id1]  # retourner toutes les valeurs
			else:
				return self.data[self.data.id == id1][X].to_dict()  # retourner les valeurs selectionnées

		else:
			if X == [] or X == self.data.keys():
				return self.data[self.data.id == id1], self.data[self.data.id == id2]  # retourner toutes les valeurs
			else:
				return self.data[self.data.id == id1][X].to_dict(), self.data[self.data.id == id2][X].to_dict()

	def V(self):
		return self.__domains.keys()


In [7]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import numpy as np
import pandas as pd

WINDOW_WIDTH = 1200
WINDOW_HEIGHT = 800

variable_entry = None
variable_type_var = None
min_entry = None
max_entry = None
step_entry = None
variables_tree = None
variable_counter = 0  # Counter pour variable names

min_label = None
max_label = None
step_label = None

def import_csv():
	file_path = filedialog.askopenfilename()
	if file_path:
		select_variables_window()

def generate_random():
	select_variables_window()

def add_variable():
	global variable_entry, variable_type_var, min_entry, max_entry, step_entry, variable_counter, variables_tree

	toggle_min_max_step_visibility()
	suggested_name = f"x{variable_counter + 1}"  # variables x1, x2, x3 ... par défaut
	variable_name = variable_entry.get() or suggested_name
	variable_type = variable_type_var.get()
	
	if variable_type == "int":
			step_value = get_default_step_value("int")
			min_value = min_entry.get() if min_entry.get() else get_default_min_value(variable_type)
			max_value = max_entry.get() if max_entry.get() else get_default_max_value(variable_type)
	elif variable_type == "boolean":
			step_value = get_default_step_value("boolean")
			min_value = get_default_min_value("boolean")
			max_value = get_default_max_value("boolean")
	else:
		min_value = min_entry.get() if min_entry.get() else get_default_min_value(variable_type)
		max_value = max_entry.get() if max_entry.get() else get_default_max_value(variable_type)
		step_value = step_entry.get() if step_entry.get() else get_default_step_value(variable_type)

	if variable_name:
		variable_counter += 1
		variable_entry.delete(0, tk.END)
		variable_entry.insert(0, f"x{variable_counter + 1}") 
		min_entry.delete(0, tk.END)
		min_entry.insert(0, get_default_min_value("int"))  
		max_entry.delete(0, tk.END)
		max_entry.insert(0, get_default_max_value("int")) 
		step_entry.delete(0, tk.END)
		step_entry.insert(0, get_default_step_value("int")) 
		variables_tree.insert("", "end", values=(variable_name, variable_type, min_value, max_value, step_value))

	toggle_min_max_step_visibility()



def update_variable_type(var_type = variable_type_var):
	global variable_type_var
	toggle_min_max_step_visibility()
	variable_type_var.set(variable_type_var.get())



def select_variables_window():
	global variable_entry, variable_type_var, min_entry, max_entry, step_entry, num_rows_entry, variables_tree, variables_window
	global min_label, max_label, step_label, next_button

	variables_window = tk.Toplevel(root)
	variables_window.title("Select Variables")
	variables_window.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")

	variable_label = tk.Label(variables_window, text="Variable Name:")
	variable_label.grid(row=0, column=0, padx=5, pady=5)
	suggested_name = f"x{variable_counter + 1}" 
	variable_entry = tk.Entry(variables_window)
	variable_entry.insert(0, suggested_name)  
	variable_entry.grid(row=0, column=1, padx=5, pady=5)

	# variable type
	type_label = tk.Label(variables_window, text="Variable Type:")
	type_label.grid(row=0, column=2, padx=5, pady=5)
	variable_type_var = tk.StringVar(variables_window)
	variable_type_var.set("int")  # Default type
	type_dropdown = tk.OptionMenu(variables_window, variable_type_var, "int", "float", "boolean", command=update_variable_type)
	type_dropdown.grid(row=0, column=3, padx=5, pady=5)

	#  min value
	min_label = tk.Label(variables_window, text="Min:")
	min_label.grid(row=0, column=4, padx=5, pady=5)
	min_entry = tk.Entry(variables_window)
	min_entry.grid(row=0, column=5, padx=5, pady=5)

	# max value 
	max_label = tk.Label(variables_window, text="Max:")
	max_label.grid(row=0, column=6, padx=5, pady=5)
	max_entry = tk.Entry(variables_window)
	max_entry.grid(row=0, column=7, padx=5, pady=5)

	# step value 
	step_label = tk.Label(variables_window, text="Arrond:")
	step_label.grid(row=0, column=8, padx=5, pady=5)
	step_entry = tk.Entry(variables_window)
	step_entry.grid(row=0, column=9, padx=5, pady=5)

	#  add variable
	add_button = tk.Button(variables_window, text="Add", command=add_variable)
	add_button.grid(row=0, column=10, padx=5, pady=5)

	# Treeview
	variables_tree = ttk.Treeview(variables_window, columns=("Name", "Type", "Min", "Max", "Step"), show="headings")
	variables_tree.heading("Name", text="Name")
	variables_tree.heading("Type", text="Type")
	variables_tree.heading("Min", text="Min")
	variables_tree.heading("Max", text="Max")
	variables_tree.heading("Step", text="Step")
	variables_tree.grid(row=1, column=0, columnspan=11, padx=5, pady=5)
	
	num_rows_label = tk.Label(variables_window, text="N:")
	num_rows_label.grid(row=2, column=7, padx=5, pady=5)
	num_rows_entry = tk.Entry(variables_window)
	num_rows_entry.insert(0, "10")  # Default number of rows
	num_rows_entry.grid(row=2, column=8, padx=5, pady=5)
	
	# add variable (pourquoi pas deux?)
	add_button2 = tk.Button(variables_window, text="Add", command=add_variable)
	add_button2.grid(row=2, column=9, padx=5, pady=5)

	# generer
	next_button = tk.Button(variables_window, text="Generate", command= lambda: display_data(get_variable_data()[0], get_variable_data()[1]))
	next_button.grid(row=2, column=10, columnspan=11, padx=5, pady=5)

	toggle_min_max_step_visibility()
	

def get_variable_data():
	variable_data = []
	for item in variables_tree.get_children():
		name = variables_tree.item(item, "values")[0]
		type = variables_tree.item(item, "values")[1]
		min_value = variables_tree.item(item, "values")[2]
		max_value = variables_tree.item(item, "values")[3]
		step_value = variables_tree.item(item, "values")[4]
		variable_data.append({"name": name, "type": type, "min": min_value, "max": max_value, "step": step_value})
	return [variable_data, int(num_rows_entry.get()) if num_rows_entry.get() else 10]


def display_data(variable_data, N):
	data_object = Data(variable_data, N)
	df = data_object.data

	data_window = tk.Toplevel(root)
	data_window.title("Data Table")
	data_window.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")
	frame = tk.Frame(data_window)
	frame.grid(sticky='nsew')

	v_scrollbar = ttk.Scrollbar(frame, orient="vertical")
	v_scrollbar.grid(row=0, column=1, sticky='ns')
	h_scrollbar = ttk.Scrollbar(frame, orient="horizontal")
	h_scrollbar.grid(row=1, column=0, sticky='ew')

	table = ttk.Treeview(frame, height=20, columns=["Index"] + [variable['name'] for variable in variable_data],
						yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set, show="headings")
	table.grid(row=0, column=0, sticky='nsew')

	table.heading("Index", text="Index")
	table.column("Index", width=100)  # Set column width to 100

	for variable in variable_data:
		table.heading(variable['name'], text=variable['name'])
		table.column(variable['name'], width=100)  # Set column width to 100

	for index, row in df.iterrows():
		table.insert("", "end", values=list(row))

	v_scrollbar.config(command=table.yview)
	h_scrollbar.config(command=table.xview)
	table.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)
	frame.grid_rowconfigure(0, weight=1)
	frame.grid_columnconfigure(0, weight=1)
	data_window.grid_columnconfigure(0, weight=1)
	data_window.grid_rowconfigure(0, weight=1)

	toggle_min_max_step_visibility()



def toggle_min_max_step_visibility():
	global min_label, max_label, step_label, min_entry, max_entry, step_entry
	selected_type = variable_type_var.get()

	if selected_type in ["int", "float"]:
		min_label.grid()
		min_entry.grid()
		max_label.grid()
		max_entry.grid()

		if selected_type == "float":
			step_label.grid()
			step_entry.grid()
		else:
			
			step_label.grid_remove()
			step_entry.grid_remove()
	else:
		min_label.grid_remove()
		min_entry.grid_remove()
		max_label.grid_remove()
		max_entry.grid_remove()
		step_label.grid_remove()
		step_entry.grid_remove()

def get_default_min_value(var_type):
	if var_type == "boolean":
		return "False"
	elif var_type == "int":
		return "0"
	elif var_type == "float":
		return "0.0"

def get_default_max_value(var_type):
	if var_type == "boolean":
		return "True"
	elif var_type == "int":
		return "100"
	elif var_type == "float":
		return "100.0"

def get_default_step_value(var_type):
	if var_type == "float":
		return "2"
	else:
		return ""






# mon gui principal
root = tk.Tk()
root.title("Bifact")
root.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")

import_button = tk.Button(root, text="Import a CSV File", command=import_csv)
import_button.pack()
generate_button = tk.Button(root, text="Generate Randomly", command=generate_random)
generate_button.pack()

root.mainloop()


,id,x1,x2,x3,y
0,0,14,23,69,
1,1,13,23,69,
2,2,13,23,69,
3,3,13,23,68,
4,4,14,23,69,
5,5,13,24,69,
6,6,13,23,69,
7,7,13,23,69,
8,8,14,23,70,
9,9,14,23,69,


In [10]:
class Bifact:
	def __init__(self,f, data):
		self.data = data
		self.f = f
	
	def AC1(self, id, X, y):
	#ligne existe, la décision est bonne et toutes les variables de X ont les bonnes valeurs aussi
		df = self.data.data
		ligne = df[df.id == id].copy()
		return len(ligne) == 1 and  ligne.y.iloc[0] == y and all((str(ligne.loc[:, [a]].iloc[0, 0]) == str(X[a])) for a in X)


	def AC2(self, id, X, y):
		df=self.data.data
		ligne=df[df.id==id].copy()
		if (len(ligne)==0):
			return False
		vals=ligne.iloc[0].to_dict()
		X_var=list(X.keys())

		variable_domains = self.data.get_domains()

		def subdict_combinations(dictionary):
			if not dictionary:
				return [{}]

			key = next(iter(dictionary))
			rest = dictionary.copy()
			del rest[key]

			subcombos = subdict_combinations(rest)
			result = []
			for subcombo in subcombos:
				result.append(subcombo)
				with_key = {key: dictionary[key]}
				result.append({**with_key, **subcombo})
			return result

		def generate_combinations(X, variable_domains):
			# Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
			combinations = product(*[variable_domains[var] for var in X])

			# Exclure la combinaison identique à X
			return [combination for combination in combinations if combination != tuple(X.values())]

		combinations = generate_combinations(X, variable_domains)

		sub=subdict_combinations(X)
		for subX in sub:
			if len(subX):
				X_var=list(subX.keys())
				vals=ligne.iloc[0].to_dict()
				combinations = generate_combinations(subX, variable_domains)
				# Vérifier chaque combinaison
				for combination in combinations:
					x_prime = dict(zip(X_var, combination))
					for k in x_prime.keys():
						vals[k]=x_prime[k]
					yhat=self.f(**{k: v for k, v in vals.items() if k in self.data.V()})

					if yhat!=y and len(x_prime)==len(X):
						print("AC2 et AC3 est vérifié pour une affectation x_prime:",x_prime, " la decision est :", yhat)
						return True
					elif yhat!=y:
						print("Erreur : AC2 et AC3 est vérifié pour une affectation x_prime :", x_prime, " la decision est :", yhat)
						return False
  
		print("Erreur : AC2 n'est pas vérifiée, veuillez saisir un superset de l'ensemble des variables sélectionné")
		return False
#ca ameliore les cas où AC3 n'est pas vérifié tout en ayant une meme complexité dans le cas où AC2 et AC3 sont verifiés

	def BC(self):
		# le code des BCi
		return True


In [ ]:
class Bifact:
	def __init__(self, f  ,data, computeY = False):
		self.data = data
		self.f = f
		if computeY:
			self.y = self.f(data)
			self.computeY = True

	def generate_causes(self, id1, id2):
     #pseudo code
		if self.computeY:
			y1, y2 = self.y[id1], self.y[id2]
		else:
			y1, y2 = self.f(self.data[id1]),self.f(self.data[id2])
		for i in range(1, len(self.data.__domains)):
			for combine in self.data.V():
				cause = Cause(id1, id2,combine)
				if cause.BC():
					return True
		return False